In [1]:
import re
from collections import Counter, OrderedDict
from dataclasses import dataclass
from functools import partial
from math import ceil, floor, prod
from itertools import combinations
from typing import Any, Dict, List, Tuple

import numpy as np

## Utils

In [2]:
def read(path):
    with open(path) as f: return f.read()

In [3]:
def mapl(func, iterable, level=1):
    if level == 0: return func(iterable)
    return [mapl(func, i, level - 1) for i in iterable]

In [4]:
a = [[[0, 1, 2], [3]], [[4]]]
assert mapl(str, a, 3) == [[["0", "1", "2"], ["3"]], [["4"]]]
assert mapl(tuple, a, 2) == [[(0, 1, 2), (3,)], [(4,)]]
assert mapl(tuple, a, 1) == [([0, 1, 2], [3]), ([4],)]
assert mapl(tuple, a, 0) == ([[0, 1, 2], [3]], [[4]])

In [5]:
def recursive_split(text, pats, level=0):
    if not pats: return text
    return recursive_split(mapl(partial(re.split, pats[0]), text, level), pats[1:], level+1)

In [6]:
assert recursive_split("text", []) == "text"
assert recursive_split("t\ne\nx\nt", ["\n"]) == ["t", "e", "x", "t"]
assert recursive_split("t\ne\n\nx\nt", ["\n\n", "\n"]) == [["t", "e"], ["x", "t"]]

In [7]:
def split(text, pats=("\n",)):
    return recursive_split(text.strip(), pats)

## Day 1

In [8]:
def split_ints(text): return mapl(int, split(text))

In [9]:
DAY01_TEST = """
1721
979
366
299
675
1456
"""
DAY01_TEST = split_ints(DAY01_TEST)
DAY01_DATA = split_ints(read("data/day01.txt"))

In [10]:
def prod_combinations_where_sum_eq(ints, n):
    return next(prod(x) for x in combinations(ints, r=n) if sum(x) == 2020)

In [11]:
assert prod_combinations_where_sum_eq(DAY01_TEST, 2) == 514579

In [12]:
prod_combinations_where_sum_eq(DAY01_DATA, 2)

866436

In [13]:
assert prod_combinations_where_sum_eq(DAY01_TEST, 3) == 241861950

In [14]:
prod_combinations_where_sum_eq(DAY01_DATA, 3)

276650720

## Day 2

In [15]:
DAY02_TEST = """
1-3 a: abcde
1-3 b: cdefg
2-9 c: ccccccccc
"""
DAY02_TEST = split(DAY02_TEST)
DAY02_DATA = split(read("data/day02.txt"))

In [16]:
def count_policy(low, high, char, password):
    return int(low) <= password.count(char) <= int(high)

In [17]:
def position_policy(low, high, char, password):
    return sum(password[int(i)-1] == char for i in (low, high)) == 1

In [18]:
def is_valid(line, policy):
    args = re.match(r"^(\d+)-(\d+) ([a-z]): ([a-z]+)$", line).groups()
    return policy(*args)

In [19]:
def count_valid_passwords(lines, policy):
    return sum(is_valid(l, policy) for l in lines)

In [20]:
assert count_valid_passwords(DAY02_TEST, count_policy) == 2

In [21]:
count_valid_passwords(DAY02_DATA, count_policy)

655

In [22]:
assert count_valid_passwords(DAY02_TEST, position_policy) == 1

In [23]:
count_valid_passwords(DAY02_DATA, position_policy)

673

## Day 3

In [24]:
DAY03_TEST = """
..##.......
#...#...#..
.#....#..#.
..#.#...#.#
.#...##..#.
..#.##.....
.#.#.#....#
.#........#
#.##...#...
#...##....#
.#..#...#.#
"""
DAY03_TEST = split(DAY03_TEST)
DAY03_DATA = split(read("data/day03.txt"))

In [25]:
def count_trees(lines, dx, dy):
    arr = np.array(mapl(lambda x: x == "#", lines, 2))
    ny, nx = arr.shape
    y = np.arange(0, ny, dy)
    x = np.mod(np.arange(0, len(y)) * dx, nx)
    return np.sum(arr[y, x])

In [26]:
SLOPES = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]

def prod_count_trees_for_slopes(lines):
    return prod(count_trees(lines, dx, dy) for dx, dy in SLOPES)

In [27]:
assert count_trees(DAY03_TEST, 3, 1) == 7

In [28]:
count_trees(DAY03_DATA, 3, 1)

216

In [29]:
EXPECTED = [2, 7, 3, 4, 2]

for (dx, dy), expected in zip(SLOPES, EXPECTED):
    assert count_trees(DAY03_TEST, dx, dy) == expected

In [30]:
assert prod_count_trees_for_slopes(DAY03_TEST) == 336

In [31]:
prod_count_trees_for_slopes(DAY03_DATA)

6708199680

## Day 4

In [32]:
def parse_passports(text):
    passports = split(text, ("\n\n", " |\n", ":"))
    return mapl(dict, passports)

In [33]:
DAY04_TEST = """
ecl:gry pid:860033327 eyr:2020 hcl:#fffffd
byr:1937 iyr:2017 cid:147 hgt:183cm

iyr:2013 ecl:amb cid:350 eyr:2023 pid:028048884
hcl:#cfa07d byr:1929

hcl:#ae17e1 iyr:2013
eyr:2024
ecl:brn pid:760753108 byr:1931
hgt:179cm

hcl:#cfa07d eyr:2025 pid:166559648
iyr:2011 ecl:brn hgt:59in
"""
DAY04_TEST = parse_passports(DAY04_TEST)
DAY04_DATA = parse_passports(read("data/day04.txt"))

In [34]:
REQUIRED = set("byr iyr eyr hgt hcl ecl pid".split())

def is_passport_keys_valid(p):
    return set(p.keys()).issuperset(REQUIRED)

In [35]:
def count_valid_passports(passports, is_valid):
    return sum(is_valid(b) for b in passports)

In [36]:
VALID_ECL = set("amb blu brn gry grn hzl oth".split())

def is_passport_valid(p):
    if not is_passport_keys_valid(p): return False
    if not (match := re.match("^(\d+)(cm|in)$", p["hgt"])): return False
    height, unit = match.groups()
    height = int(height)
    valid = {
        "byr": 1920 <= int(p["byr"]) <= 2002,
        "iyr": 2010 <= int(p["iyr"]) <= 2020,
        "eyr": 2020 <= int(p["eyr"]) <= 2030,
        "hgt": (150 <= height <= 193 if unit == "cm" else 59 <= height <= 76),
        "hcl": re.match("^\#[0-9a-f]{6}$", p["hcl"]) is not None,
        "ecl": p["ecl"] in VALID_ECL,
        "pid": re.match("^\d{9}$", p["pid"]) is not None
    }
    return all(valid.values())

In [37]:
assert count_valid_passports(DAY04_TEST, is_passport_keys_valid) == 2

In [38]:
count_valid_passports(DAY04_DATA, is_passport_keys_valid)

210

In [39]:
INVALID_PASSPORTS = parse_passports(
"""
eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007
"""
)
VALID_PASSPORTS = parse_passports(
"""
pid:087499704 hgt:74in ecl:grn iyr:2012 eyr:2030 byr:1980
hcl:#623a2f

eyr:2029 ecl:blu cid:129 byr:1989
iyr:2014 pid:896056539 hcl:#a97842 hgt:165cm

hcl:#888785
hgt:164cm byr:2001 iyr:2015 cid:88
pid:545766238 ecl:hzl
eyr:2022

iyr:2010 hgt:158cm hcl:#b6652a ecl:blu byr:1944 eyr:2021 pid:093154719
"""
)

In [40]:
for p in VALID_PASSPORTS:
    assert is_passport_valid(p), p

In [41]:
for p in INVALID_PASSPORTS:
    assert not is_passport_valid(p), p

In [42]:
count_valid_passports(DAY04_DATA, is_passport_valid)

131

## Day 5

In [43]:
def decode(chars, low, up):
    for char in chars:
        mid = (low + up) / 2
        if char in "FL": up  = floor(mid)
        else:            low = ceil(mid)
    return low

def parse_seat_id(line):
    row, col = decode(line[:7], 0, 127), decode(line[7:], 0, 7)
    return 8 * row + col

def parse_seat_ids(text):
    return mapl(parse_seat_id, split(text))

In [44]:
DAY05_TEST = """
FBFBBFFRLR
BFFFBBFRRR
FFFBBBFRRR
BBFFBBFRLL
"""
DAY05_TEST = parse_seat_ids(DAY05_TEST)
DAY05_DATA = parse_seat_ids(read("data/day05.txt"))

In [45]:
TEST_SEAT_IDS = [357, 567, 119, 820]
for sid, exp in zip(DAY05_TEST, TEST_SEAT_IDS):
    assert sid == exp

In [46]:
max(DAY05_DATA)

913

In [47]:
set(range(min(DAY05_DATA), max(DAY05_DATA) + 1)) - set(DAY05_DATA)

{717}

## Day 6

In [48]:
def parse_customs_forms(text):
    forms = split(text, ("\n\n", "\n"))
    return mapl(set, forms, 2)

In [49]:
DAY06_TEST = """
abc

a
b
c

ab
ac

a
a
a
a

b
"""
DAY06_TEST = parse_customs_forms(DAY06_TEST)
DAY06_DATA = parse_customs_forms(read("data/day06.txt"))

In [50]:
def count_answers(forms, set_agg):
    return sum(len(set_agg(*x)) for x in forms)

In [51]:
assert count_answers(DAY06_TEST, set.union) == 11

In [52]:
count_answers(DAY06_DATA, set.union)

6930

In [53]:
assert count_answers(DAY06_TEST, set.intersection) == 6

In [54]:
count_answers(DAY06_DATA, set.intersection)

3585

## Day 7

In [55]:
# DAY07_TEST = """
# """
# DAY07_TEST = split(DAY07_TEST)
# DAY07_DATA = split(read("data/day07.txt"))